In [1]:
def importdata():
    
    df = pd.read_csv('../data/df_points_valid_geo_v100.csv')

    df = df[['bike_id','lat','lng','timestood','time_start','time_end']]
    
    #df['date_start'] = df['time_start']
    #df['date_end'] = df['time_end']
    #df['daytime_start'] = df['time_start']
    #df['daytime_end'] = df['time_end']
    
    df['flag'] = 0
    
    
    
    return df

# Main

In [3]:
from keplergl import KeplerGl
import pandas as pd
pd.options.mode.chained_assignment = None
from tqdm import tqdm
import pickle
from datetime import timedelta, datetime, date


df = importdata()


df


,bike_id,lat,lng,timestood,time_start,time_end,flag
0,71949.0,51.334071,12.399960,5238.0,1.546298e+09,1.546303e+09,0
1,71949.0,51.335429,12.324583,7351.0,1.546307e+09,1.546314e+09,0
2,71949.0,51.343813,12.381495,35539.0,1.546316e+09,1.546352e+09,0
3,71949.0,51.341518,12.365614,198912.0,1.546352e+09,1.546551e+09,0
4,71949.0,51.345950,12.356816,45642.0,1.546551e+09,1.546597e+09,0
...,...,...,...,...,...,...,...
550860,75870.0,51.329036,12.339146,1580.0,1.577631e+09,1.577632e+09,0
550861,75870.0,51.320889,12.343077,61152.0,1.577632e+09,1.577694e+09,0
550862,75870.0,51.332209,12.366046,30954.0,1.577694e+09,1.577725e+09,0
550863,105465.0,0.000000,0.000000,64439.0,1.575910e+09,1.575975e+09,0


In [4]:
df['timestood'] = pd.to_timedelta(df['timestood'],unit='s')
df['time_start'] = pd.to_datetime(df['time_start'],unit='s')
df['time_end'] = pd.to_datetime(df['time_end'],unit='s')


df

,bike_id,lat,lng,timestood,time_start,time_end,flag
0,71949.0,51.334071,12.399960,0 days 01:27:18,2018-12-31 23:14:49,2019-01-01 00:42:07,0
1,71949.0,51.335429,12.324583,0 days 02:02:31,2019-01-01 01:41:33,2019-01-01 03:44:04,0
2,71949.0,51.343813,12.381495,0 days 09:52:19,2019-01-01 04:14:54,2019-01-01 14:07:13,0
3,71949.0,51.341518,12.365614,2 days 07:15:12,2019-01-01 14:13:45,2019-01-03 21:28:57,0
4,71949.0,51.345950,12.356816,0 days 12:40:42,2019-01-03 21:35:13,2019-01-04 10:15:55,0
...,...,...,...,...,...,...,...
550860,75870.0,51.329036,12.339146,0 days 00:26:20,2019-12-29 14:42:51,2019-12-29 15:09:11,0
550861,75870.0,51.320889,12.343077,0 days 16:59:12,2019-12-29 15:13:29,2019-12-30 08:12:41,0
550862,75870.0,51.332209,12.366046,0 days 08:35:54,2019-12-30 08:23:51,2019-12-30 16:59:45,0
550863,105465.0,0.000000,0.000000,0 days 17:53:59,2019-12-09 16:49:55,2019-12-10 10:43:54,0


In [7]:
from tqdm import tqdm

changed = 1
iterations = 0

while changed:
    iterations += 1
    changed = 0
    for ind in tqdm(df.index):
        ts = df['time_start'][ind]
        te = df['time_end'][ind]
        t0 = ts.replace(hour=23, minute=59, second=59)
        
        # is timestood greater than same date 23:59:59
        #   -> over midnight
        if ts + df['timestood'][ind] > t0:
            changed = 1
            
            underhang = t0 - ts
            overhang = te - t0
            
            # fix current day
            df['time_end'][ind] = ts.replace(hour=23, minute=59, second=59)
            df['timestood'][ind] = underhang
            
            # add new row
            time_start = ts.replace(hour=0, minute=0, second=0)
            time_start += timedelta(days=1)
            
            flag = 0
            if time_start + overhang > time_start.replace(hour=23, minute=59, second=59):
                flag = 1
            
            dict = {'bike_id': df['bike_id'][ind], 'lng': df['lng'][ind], 'lat': df['lat'][ind],
                    'timestood': overhang, 'time_start': time_start, 'time_end': te, 'flag' : flag}
            df = df.append(dict, ignore_index=True)
print(iterations)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 703444/703444 [00:28<00:00, 24461.98it/s]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 703614/703614 [00:27<00:00, 25597.11it/s]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 703709/703709 [00:27<00:00, 25847.53it/s]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 703716/703716 [00:27<00:00, 25843.48it/s]

211


In [10]:
df

,bike_id,lat,lng,timestood,time_start,time_end,flag
0,71949.0,51.334071,12.399960,0 days 00:45:10,2018-12-31 23:14:49,2018-12-31 23:59:59,0
1,71949.0,51.335429,12.324583,0 days 02:02:31,2019-01-01 01:41:33,2019-01-01 03:44:04,0
2,71949.0,51.343813,12.381495,0 days 09:52:19,2019-01-01 04:14:54,2019-01-01 14:07:13,0
3,71949.0,51.341518,12.365614,0 days 09:46:14,2019-01-01 14:13:45,2019-01-01 23:59:59,0
4,71949.0,51.345950,12.356816,0 days 02:24:46,2019-01-03 21:35:13,2019-01-03 23:59:59,0
...,...,...,...,...,...,...,...
703711,41802.0,51.347019,12.384081,0 days 23:59:59,2019-08-28 00:00:00,2019-08-28 23:59:59,1
703712,41802.0,51.347019,12.384081,0 days 23:59:59,2019-08-29 00:00:00,2019-08-29 23:59:59,1
703713,41802.0,51.347019,12.384081,0 days 23:59:59,2019-08-30 00:00:00,2019-08-30 23:59:59,1
703714,41802.0,51.347019,12.384081,0 days 23:59:59,2019-08-31 00:00:00,2019-08-31 23:59:59,1


In [21]:
# Fix timestood to minutes in int
for ind in tqdm(df.index):
    df['timestood'][ind] = df['timestood'][ind].seconds / 60

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 703716/703716 [1:03:48<00:00, 183.79it/s]


In [22]:
df[['timestood']] = df[['timestood']].astype(int)
df

,bike_id,lat,lng,timestood,time_start,time_end,flag
0,71949.0,51.334071,12.399960,45,2018-12-31 23:14:49,2018-12-31 23:59:59,0
1,71949.0,51.335429,12.324583,122,2019-01-01 01:41:33,2019-01-01 03:44:04,0
2,71949.0,51.343813,12.381495,592,2019-01-01 04:14:54,2019-01-01 14:07:13,0
3,71949.0,51.341518,12.365614,586,2019-01-01 14:13:45,2019-01-01 23:59:59,0
4,71949.0,51.345950,12.356816,144,2019-01-03 21:35:13,2019-01-03 23:59:59,0
...,...,...,...,...,...,...,...
703711,41802.0,51.347019,12.384081,1439,2019-08-28 00:00:00,2019-08-28 23:59:59,1
703712,41802.0,51.347019,12.384081,1439,2019-08-29 00:00:00,2019-08-29 23:59:59,1
703713,41802.0,51.347019,12.384081,1439,2019-08-30 00:00:00,2019-08-30 23:59:59,1
703714,41802.0,51.347019,12.384081,1439,2019-08-31 00:00:00,2019-08-31 23:59:59,1


# Save CSV 

In [23]:
df.to_csv("../data/df_points_valid_geo_split_at_midnight_v100.csv",index=False)

In [24]:
df['timestood'].max()

1439


### Loop | rows | max(idle_time)

0 | 550670

1 | 667360 | 209 days 01:14:44

2 | 683917 | 208 days 01:14:44

3 | 690781 | 207 days 01:14:44

4 | 694443 | 206 days 01:14:44

...

x | 703716 | 0 days 23:59:59